In [12]:
import sys
deriva_py = '/Users/carl/Repos/deriva-py/deriva_common'

sys.path
sys.path.extend([deriva_py])


In [5]:
import tempfile
import pandas as pd
import numpy as np

import os
import shutil

from deriva_common import HatracStore, ErmrestCatalog, get_credential

import IPython.core.debugger

Set up debugging and logging.

In [ ]:
dbg = IPython.core.debugger.Pdb
from IPython.core.debugger import set_trace

# Configuring the logger for debug level will display the uri's generated by the api
debug = False
if debug:
    import logging
    logger = logging.getLogger('deriva_common.datasets')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    logger.addHandler(ch)

In [ ]:
def get_synapses(synapsefiles, study):
   
    '''
     synapsefile: a HatracStore object
     study: a dictionary that has URLs for the two images, before and after
     
     returns two pandas that have the synapses in them.
     ''' 

    try:
        # Get a path for a tempory file to store HATRAC results 
        path = os.path.join(tempfile.mkdtemp(), 'image')
        
        # Get the before image from hatrac, be careful in case its missing
        if study['Region 1 URL']:
            synapsefiles.get_obj(study['Region 1 URL'], destfilename=path)
            img1 = pd.read_csv(path)
            img1.drop(img1.index[0],inplace=True)
        else:
            img1 = None
        
        # Get the after image from hatrac, be careful in case its missing
        if study['Region 2 URL']:
            synapsefiles.get_obj(study['Region 2 URL'], destfilename=path)
            img2 = pd.read_csv(path)
            img2.drop(img2.index[0],inplace=True)
        else:
            img2 = None
    finally:
        shutil.rmtree(os.path.dirname(path))
    return {'before':img1, 'after': img2, 'type': study['type']}    

In [ ]:
# Plot a study

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

def plot_study(study):
    '''
    Create a 3D scatter plot of a study.
    '''
    trace1 = go.Scatter3d(
        x=study['before']['X'],
        y=study['before']['Y'],
        z=study['before']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    trace2 = go.Scatter3d(
        x=study['after']['X'],
        y=study['after']['Y'],
        z=study['after']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [trace1, trace2]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

Connect to synapse catalog

In [43]:
# Need to use Deriva authentication agent before executing this
credential = get_credential("synapse.isrd.isi.edu")

objectstore = HatracStore('https','synapse.isrd.isi.edu',credentials=credential)
catalog = ErmrestCatalog('https','synapse.isrd.isi.edu', 1, credentials=credential)

In [83]:
pb = catalog.getPathBuilder()

# conenient name for the schema we care about.
zebrafish = pb.Zebrafish
synapse = pb.Synapse
    
#build up the path....
path = zebrafish.tables['Synaptic Pair Study'].alias('study')\
    .link(zebrafish.tables['Image Pair Study'].alias('pair'))\
    .link(zebrafish.Behavior, join_type = 'left', on=(path.pair.Subject == zebrafish.Behavior.Subject))
    
# Get the protocol from the first image...
path = path.pair.link(zebrafish.Image, on=path.pair.columns['Image 1'] == zebrafish.Image.ID)\
    .link(synapse.tables['Protocol Step'])\
    .link(synapse.Protocol)
    

study_entities = path.study.entities(path.study.Study, 
                                     path.pair.Subject, 
                                     path.Behavior.columns['Learned?'],
                                     path.Protocol.ID,
                                     path.study.columns['Region 1 URL'],
                                     path.study.columns['Region 2 URL'])

print(len(study_entities))
print(study_entities[0])


28
{'Study': 6023, 'Subject': 'ZfDsy20160116A', 'Learned?': True, 'ID': 'PrcDsy20160101A', 'Region 1 URL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s1-registered.csv:YND3C4Q2LAEPNNNAGTNTNK36RE', 'Region 2 URL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s2-registered.csv:K2FFS4UIS422GQF5EXYWA5372A'}


Turn entity set into a list, mapping the protocol name to get the study type.

In [89]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',  
    'PrcDsy20170613A': 'conditioned', 
    'PrcDsy20170615A': 'unconditioned', 
    'PrcDsy20170613B': 'control'
}

for i in study_entities:
    if protocol_types[i['ID']] == 'aversion':
        i['Type'] = 'learner' if i['Learned?'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['ID']]
    study_list.append(i)

In [ ]:
synapses = [get_synapses(objectstore, study) for study in study_list]

In [ ]:
plot_study(synapses[0])